<a href="https://colab.research.google.com/github/Hoangthang017/CS114.K21/blob/master/FaceMask/FaceMaskDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [35]:
# import các thư viện cần dùng 

# kỹ thuật data aug
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Sử dụng fine-turning model MobileNetV2
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


# thư viện để pre-process data
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer

# thư viện để chia dữ liệu
from sklearn.model_selection import train_test_split

# thư viện để đánh giá model
from sklearn.metrics import classification_report

# load data 
from imutils import paths
import numpy as np
import os


In [11]:
# load dataset 
print("[INFO] loading images...")
imagePaths = list(paths.list_images('/content/drive/My Drive/CS114/FaceMask/myDataset'))

# khởi tạo dữ liệu
data = []

# khởi tạo label ứng với dữ liệu
labels = []

[INFO] loading images...


In [12]:
# Vòng lặp để load dữ liệu

for imagePath in imagePaths:
	# lấy nhãn từ đường dẫn tệp
	label = imagePath.split(os.path.sep)[-2]

	# load hình ảnh ở dạng PIL(Python Imaging Library) 
	# Thay đổi kích thước ảnh đầu vào dạng 244 x 244 pixels
	image = load_img(imagePath, target_size=(224, 224))
 
	# chuyển đổi hình ảnh từ PIL sang dạng mảng
	image = img_to_array(image)

	# pre-process ảnh để đẩy vào mạng MobileNetV2
	image = preprocess_input(image)
 
 	# đẩy dữ liệu vào data và label 
	data.append(image)
	labels.append(label)


/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


In [13]:
print(len(data))

3667


In [14]:
# chuyển dữ liệu từ hình ảnh và label sang dạng mảng
data = np.array(data, dtype="float32")
labels = np.array(labels)
print(data[0])
print(labels)

[[[-0.42745095 -0.38823527 -0.23921567]
  [-0.42745095 -0.38823527 -0.23921567]
  [-0.42745095 -0.38823527 -0.23921567]
  ...
  [ 0.34901965  0.4039216   0.5294118 ]
  [ 0.34901965  0.4039216   0.5294118 ]
  [ 0.34901965  0.4039216   0.5294118 ]]

 [[-0.42745095 -0.38823527 -0.23921567]
  [-0.42745095 -0.38823527 -0.23921567]
  [-0.42745095 -0.38823527 -0.23921567]
  ...
  [ 0.34901965  0.4039216   0.5294118 ]
  [ 0.34901965  0.4039216   0.5294118 ]
  [ 0.34901965  0.4039216   0.5294118 ]]

 [[-0.41960782 -0.38039213 -0.23137254]
  [-0.41960782 -0.38039213 -0.23137254]
  [-0.41960782 -0.38039213 -0.23137254]
  ...
  [ 0.3176471   0.37254906  0.5137255 ]
  [ 0.3176471   0.37254906  0.5137255 ]
  [ 0.3176471   0.37254906  0.5137255 ]]

 ...

 [[-0.88235295 -0.92156863 -0.9529412 ]
  [-0.88235295 -0.92156863 -0.9529412 ]
  [-0.88235295 -0.92156863 -0.9529412 ]
  ...
  [-0.52156866 -0.6        -0.5921569 ]
  [-0.52156866 -0.6        -0.5921569 ]
  [-0.52156866 -0.6        -0.5921569 ]]

 [

In [15]:
# mã hóa label từ dạng chữ thành số 
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

In [16]:
# chia đữ liệu thành 80% để train và 20% để test
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.20, stratify=labels, random_state=42)
print(len(trainX))
print(trainX.shape)

2933
(2933, 224, 224, 3)


In [34]:
# Sử dụng kỹ thuật Data Augmentation để tăng kích thước cho bộ train data
# fine-turning hàm ImageDataGenerator
aug = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,)

# **Fine-turning model MobileNetv2 và training cho model**

In [28]:
# các tham số để cài đặt trong model

# Tốc độ học
INIT_LR = 1e-4

# số lần pre-train 
EPOCHS = 15

# số lượng mẫu dữ liệu trong 1 batch 
BS = 32

- weights="imagenet": pre-training trên ImageNet
- include_top=False:  Bỏ đi các lớp kết nối ở đầu mạng
- input_tensor=Input(shape=(224, 224, 3)): cho đầu vào hình ảnh của model ở dạng (224, 224, 3)

In [19]:
# load model MobileNetV2 
baseModel = MobileNetV2(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))

9412608/9406464 [==============================] - 0s 0us/step


In [20]:
# fine-turning cho model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

In [21]:
# đưa model đã fine-turning vào model cơ sở
model = Model(inputs=baseModel.input, outputs=headModel)

In [22]:
# đóng băng tất cả các lớp trong model cơ sở để chúng không cập nhật trong quá trình train
for layer in baseModel.layers:
	layer.trainable = False

In [23]:
# cấu hình model cho training
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

In [29]:
# Thực hiện việc train 
print("[INFO] training head...")
H = model.fit(
	aug.flow(trainX, trainY, batch_size=BS),
	steps_per_epoch=len(trainX) // BS,
	validation_data=(testX, testY),
	validation_steps=len(testX) // BS,
	epochs=EPOCHS)

[INFO] training head...
Epoch 1/15
91/91 [==============================] - 143s 2s/step - loss: 0.0235 - accuracy: 0.9931 - val_loss: 0.0299 - val_accuracy: 0.9864
Epoch 2/15
91/91 [==============================] - 144s 2s/step - loss: 0.0176 - accuracy: 0.9938 - val_loss: 0.0286 - val_accuracy: 0.9864
Epoch 3/15
91/91 [==============================] - 143s 2s/step - loss: 0.0196 - accuracy: 0.9924 - val_loss: 0.0248 - val_accuracy: 0.9905
Epoch 4/15
91/91 [==============================] - 143s 2s/step - loss: 0.0223 - accuracy: 0.9931 - val_loss: 0.0266 - val_accuracy: 0.9891
Epoch 5/15
91/91 [==============================] - 144s 2s/step - loss: 0.0177 - accuracy: 0.9945 - val_loss: 0.0239 - val_accuracy: 0.9905
Epoch 6/15
91/91 [==============================] - 143s 2s/step - loss: 0.0198 - accuracy: 0.9931 - val_loss: 0.0285 - val_accuracy: 0.9837
Epoch 7/15
91/91 [==============================] - 143s 2s/step - loss: 0.0301 - accuracy: 0.9903 - val_loss: 0.0310 - val_accura

#**Đánh giá model**

In [30]:
# tiến hành predict cho tập train
print("[INFO] evaluating network...") 
predIdxs = model.predict(testX, batch_size=BS)

[INFO] evaluating network...


In [31]:
# for each image in the testing set we need to find the index of the label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)

In [32]:
# kết quả đánh giá
print(classification_report(testY.argmax(axis=1), predIdxs,
	target_names=lb.classes_))

              precision    recall  f1-score   support

   with_mask       0.98      1.00      0.99       384
without_mask       1.00      0.97      0.99       350

    accuracy                           0.99       734
   macro avg       0.99      0.99      0.99       734
weighted avg       0.99      0.99      0.99       734



##**Save model**


In [33]:
model.save('/content/drive/My Drive/CS114/FaceMask')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: /content/drive/My Drive/CS114/FaceMask/assets
